In [22]:
#Import the package to your work environment
import fingertips_py as ftp
import pandas as pd

In [63]:
# Define the Indicator ID: Depression incidence
depression = 848 

# Fetch the data for the specified indicator at all available geographies
depression_data = ftp.get_data_for_indicator_at_all_available_geographies(depression)

# Display the first few rows of the data to understand the structure
depression_data.head(2)

c:\Bruno\Python\Lib\site-packages\fingertips_py\retrieve_data.py:46: DtypeWarning: Columns (9,10,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(base_url + populated_url)


,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to percentiles,Time period Sortable,New data,Compared to goal,Time period range
0,848,Depression: QOF prevalence (18+ yrs) - retired...,NaN,NaN,E92000001,England,England,Persons,18+ yrs,NaN,...,2582233,4.423848e+07,There is a data quality issue with this value,NaN,Not compared,Not compared,20120000,NaN,NaN,1y
1,848,Depression: QOF prevalence (18+ yrs) - retired...,NaN,NaN,E92000001,England,England,Persons,18+ yrs,County & UA deprivation deciles in England (IM...,...,266115,4.253045e+06,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20120000,NaN,NaN,1y


In [66]:
# Filter the data for 'Most deprived decile'
most_deprived_data = depression_data[(depression_data['Area Name'] == 'England') &
                                     (depression_data['Parent Code'] == 'E92000001') &
                                     (depression_data['Time period range'] == '1y') &
                                     (depression_data['Category'] == 'Most deprived decile')]

# Filter the data for 'Least deprived decile'
least_deprived_data = depression_data[(depression_data['Area Name'] == 'England') &
                                      (depression_data['Parent Code'] == 'E92000001') &
                                      (depression_data['Time period range'] == '1y') &
                                      (depression_data['Category'] == 'Least deprived decile ')]

# Combine the data for visualization
combined_data = pd.concat([
    most_deprived_data.assign(Category='Most Deprived Decile'),
    least_deprived_data.assign(Category='Least Deprived Decile')
])

# Create a new 'Year' column with numeric years
combined_data['Year'] = combined_data['Time period'].str[:4].astype(int)

# Convert the 'Value' to percentage format if necessary (assuming it's a proportion)
combined_data['Value'] = combined_data['Value']/100 # Only do this if needed to convert to percentage


In [143]:
import altair as alt
import pandas as pd

# Define custom colors for the lines
custom_colors = alt.Scale(domain=['Most Deprived Decile', 'Least Deprived Decile'], 
                          range=['#17a2b8', '#fc6c85'])  # Teal and Coral

# Create the Altair line chart with updated line colors
chart = alt.Chart(combined_data).mark_line().encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0,
            labelFontSize=12,
            titleFontSize=14,
            labelColor='black',
            titleColor='black')),
    y=alt.Y(
        'Value:Q',
        title='Depression Prevalence Rate (%)',
        axis=alt.Axis(
            format='%',
            labelFontSize=12,
            titleFontSize=14,
            labelColor='black',
            titleColor='black'
        )
    ),
    color=alt.Color('Category:N', scale=custom_colors, legend=alt.Legend(title='Deprivation Decile')),
    tooltip=['Year:O', 'Value:Q', 'Category:N']
).properties(
    title=alt.TitleParams(
        text='Depression Prevalence Rate in England: Most vs Least Deprived Decile',
        subtitle='Data from the QOF (Quality and Outcomes Framework) for 18+ years',
        fontSize=16,           # Set font size for the main title
        subtitleFontSize=14,   # Set font size for the subtitle
        font='Arial',          # Customize font family (optional)
        subtitleFont='Arial'   # Customize font family for subtitle (optional)
    ),
    width=500,
    height=350
).interactive()  # Adds zoom and pan capabilities

# Add the source as a text label below the chart
source = alt.Chart(pd.DataFrame({'text': ['Source: Fingertips']})).mark_text(
    align='left',
    baseline='bottom',
    fontSize=12,
    color='black'
).encode(
    text='text:N'
).properties(
    width=-75,
    height=0
)

# Combine the chart and source text
final_chart = alt.vconcat(chart, source)
final_chart
final_chart.save('Chart_1_dep_prev.json')